# Se realizan importaciones

In [1]:
from Servicios.GoogleDriveService.GoogleDriveService import GoogleDriveService
from Servicios.AdvanaService.AdvanaService import AdvanaService
from Servicios.OrquestadorService.OrchestrateService import OrchestrateService

from Utilidades.GlobalUtility import ObtenerDataPorHTTPRequest
import Utilidades.GlobalUtility as Utility

# Se definen variables de Buckets de GCP

In [2]:
IdContenedorBronze = "1yQCQ3BhNPdXiz890KTF9iNB5qdeTATlV" #Bronce
IdContenedorGold = "1TMq41pVhsEDK1IUP4kQHzf7aIsagVihE" #Gold

URLRequestRawDataZip = "https://drive.usercontent.google.com/download?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis&export=download&authuser=0&confirm=t&uuid=d3ab3e9e-0e9e-4dfe-945a-a73286456315&at=APZUnTUiCQBVxpETIVQcpNAPpA7U%3A1720637781554"
NombreArchivoEnDataLake = "data.json"

# Se definen variables de Advana

In [3]:
JSONObjEntregaAdvana = {"name": "TDD (Pruebas) EnviarPeticionHTTP() Advana Fmateu",
           "mail": "francisco.mateu.araneda@gmail.com Aun en desarrollo",
           "github_url": "https://github.com/xFrancisc0/PrototipoDE"}
           
URLRequestEntregaAdvana = "https://advana-challenge-check-api-cr-k4hdbggvoq-uc.a.run.app/data-engineer"

# Se instancian los servicios de la aplicación y se inyectan en el orquestador
El objetivo es una vez que se instancie el orquestador, que se orqueste una ingesta de data desde https://drive.google.com/file/d/1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis/view?usp=sharing y que se guarde en la zona bronze del "Datalake" de prototipo Es una URL pública: https://drive.google.com/drive/u/2/folders/1nVbnD0pgYnAeym3lUoNna5cE7goN2U5N (Fue desplegado este Datalake para ser visto desde el exterior sin permisos de edición. Esto es para mostrar el funcionamiento del presente proyecto)

Posteriormente, el orquestador cargara la data desde la zona bronce, se le aplicarán todos los procedimientos y se guardará en la zona Gold.

In [4]:
# Crear instancias de servicios e inyectarlos en el orquestador para mayor simplicidad
google_drive_service = GoogleDriveService()
advana_service = AdvanaService()

# Crear instancia de OrchestrateService, inyectando GoogleDriveService
orchestrate_service = OrchestrateService(google_drive_service, advana_service)

#Destruir objetos que no se ocuparan
del google_drive_service
del advana_service

# Se obtiene la raw data y se almacena en zona bronze
Para ver el datalake: https://drive.google.com/drive/u/2/folders/1nVbnD0pgYnAeym3lUoNna5cE7goN2U5N

In [5]:
RawDataInput = Utility.ObtenerDataPorHTTPRequest(URLRequestRawDataZip, 'GET', 'JSON', True)

#Cargar raw data a zona bronze
orchestrate_service.google_drive_service.subir_archivo_a_contenedor(IdContenedorBronze, NombreArchivoEnDataLake, RawDataInput)

#Destruir objeto RawDataInput
del RawDataInput


Obteniendo JSON POR HTTP Request
Archivo JSON descomprimido y cargado exitosamente.
Subiendo data a Google Drive
Archivo data.json ya existente en google cloud con ID 1KZnyePYgij75NQbcBw8vcS0lzE9dwHko eliminado (overwrite).
Archivo 'data.json' subido correctamente. ID: 12QOHy_SYNNKosAkyWQ3nqanDVftUhkMX


# Se obtiene la data de la zona de bronze 

In [6]:
#Cargar en memoria de zona bronze
BronzeMetadata = orchestrate_service.google_drive_service.listar_archivos_en_contenedorSTR(IdContenedorBronze)
BronzeDataInput = orchestrate_service.google_drive_service.descargar_archivo_porid(BronzeMetadata[0]["id"])

Archivos encontrados:  [{'kind': 'drive#file', 'mimeType': 'application/json', 'id': '12QOHy_SYNNKosAkyWQ3nqanDVftUhkMX', 'name': 'data.json'}]
Descargando 25%.
Descargando 51%.
Descargando 77%.
Descargando 100%.


# Se procede a realizar las transformaciones requeridas

In [7]:
BronzeDataETL = BronzeDataInput

#Destruir objeto BronzeDataETL
del BronzeDataInput



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



# El dataset posterior a sus transformaciones se guarda en la zona gold del Datalake
Para ver el datalake: https://drive.google.com/drive/u/2/folders/1nVbnD0pgYnAeym3lUoNna5cE7goN2U5N

In [8]:
"""
orchestrate_service.google_drive_service.subir_archivo_a_contenedor(IdContenedorGold, NombreArchivoEnDataLake, BronzeDataETL)

#Destruir objeto RawDataInput
del BronzeDataETL
"""

Obteniendo JSON POR HTTP Request
Archivo JSON descomprimido y cargado exitosamente.
